In [2]:
import os
import yaml
from minio import Minio
from ultralytics import YOLO

In [3]:
client = Minio(
    "172.21.147.11:9000",
    access_key="admin",
    secret_key="password123",
    secure=False
)

try:
    buckets = client.list_buckets()
    for bucket in buckets:
        print(f"버킷: {bucket.name}")
except Exception as e:
    print(f"연결 오류: {e}")

try:
    objects = client.list_objects("ai-door-system", prefix="datasets/v1/", recursive=True)
    print("\n파일 목록:")
    for obj in objects:
        print(f"- {obj.object_name}")
except Exception as e:
    print(f"파일 목록 조회 오류: {e}")

버킷: ai-door-system

파일 목록:
- datasets/v1/.keep
- datasets/v1/data.yaml
- datasets/v1/train/images/IMG_6817_jpg.rf.0d5ba30a7d93a71ad91468bb92a8c0f6.jpg
- datasets/v1/train/images/IMG_6818_jpg.rf.569ae5fe58dfb810a91e752559a237e0.jpg
- datasets/v1/train/images/IMG_6821_jpg.rf.a18b1e0d4d83d790a2d4e89151ea2da1.jpg
- datasets/v1/train/images/IMG_6822_jpg.rf.39ca20a1ada24ba1b6524048cbc8d6cd.jpg
- datasets/v1/train/images/IMG_6823_jpg.rf.0b1de0555263d60053640d995cb42c69.jpg
- datasets/v1/train/images/IMG_6824_jpg.rf.84eee60cade94e9561de2ac2118b3c1b.jpg
- datasets/v1/train/images/PXL_20201101_154113387_jpg.rf.b87c7a58052e48724d364c961c5b9983.jpg
- datasets/v1/train/images/PXL_20201103_181906837_jpg.rf.0acd286eafa7dd45dbe43650c2bf39dd.jpg
- datasets/v1/train/images/PXL_20201103_181911592_jpg.rf.a633a6b6727ee5b8616b3654697d08bd.jpg
- datasets/v1/train/images/PXL_20201105_173148569_jpg.rf.399116b84089a4ae76af0e80f8dcacc1.jpg
- datasets/v1/train/images/PXL_20201107_021341143_jpg.rf.9f5044f4a26540d6

In [4]:
# 데이터셋 읽기--임시 폴더 생성
import tempfile

def get_temp_dataset_path():
    temp_dir = tempfile.mkdtemp(prefix="yolo_dataset_")
    return temp_dir

def download_dataset_from_minio(bucket, dataset_prefix, local_path="/tmp/dataset"):
    if local_path is None:
        local_path = get_temp_dataset_path()
    
    os.makedirs(local_path, exist_ok=True)
    
    objects = client.list_objects(bucket, prefix=dataset_prefix, recursive=True)
    
    for obj in objects:
        if obj.object_name.endswith(('.jpg', '.jpeg', '.png', '.txt', '.yaml')):
            relative_path = obj.object_name.replace(f"{dataset_prefix}/", "")
            local_file_path = os.path.join(local_path, relative_path)
            os.makedirs(os.path.dirname(local_file_path), exist_ok=True)
            client.fget_object(bucket, obj.object_name, local_file_path)
    return local_path

In [5]:
# data.yaml
dataset_path = download_dataset_from_minio("ai-door-system", "datasets/v1")
data_yaml_path = os.path.join(dataset_path, "data.yaml")

if not os.path.exists(data_yaml_path):
    print(f"data.yaml 파일을 찾을 수 없습니다: {data_yaml_path}")
else:
    print(f"data.yaml 파일: {data_yaml_path}")

data.yaml 파일: /tmp/dataset\data.yaml


In [6]:
# 사전 훈련된 모델 로드
model = YOLO('yolo11s.pt')

In [7]:
# 모델 설정 파일 로드
with open('./config.yaml', 'r') as f:
    model_config = yaml.safe_load(f)

In [8]:
# 모델 학습
training_config = model_config['training']
TRAIN_NAME = f"{training_config['model_name']}_{training_config['version']}"

results = model.train(
    data=data_yaml_path,
    imgsz=training_config['imgsz'],
    epochs=training_config['epochs'],
    batch=training_config['batch'],
    lr0=training_config['lr0'],
    
    patience=training_config['patience'],
    save_period=training_config['save_period'],
    cache=training_config['cache'],
    workers=training_config['workers'],
    
    optimizer=training_config['optimizer'],
    weight_decay=training_config['weight_decay'],
    warmup_epochs=training_config['warmup_epochs'],
    warmup_momentum=training_config['warmup_momentum'],
    warmup_bias_lr=training_config['warmup_bias_lr'],
    
    val=training_config['val'],
    split=training_config['split'],
    
    plots=training_config['plots'],
    name=TRAIN_NAME,
    task='detect',
    save=training_config['save'],
    verbose=training_config['verbose'],
)

New https://pypi.org/project/ultralytics/8.3.176 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.173  Python-3.11.13 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i7-12700)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/tmp/dataset\data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=package-detection_v1.0.07, nbs=64, nms=False, opset

train: Scanning C:\tmp\dataset\train\labels.cache... 19 images, 0 backgrounds, 0 corrupt: 100%|██████████| 19/19 [00:00<?, ?it/s]

val: Fast image access  (ping: 0.00.0 ms, read: 1980.7873.1 MB/s, size: 772.1 KB)



c:\Users\SBA\miniconda3\envs\project\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
val: Scanning C:\tmp\dataset\val\labels.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
c:\Users\SBA\miniconda3\envs\project\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


optimizer: SGD(lr=0.001, momentum=0.937) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 320 train, 320 val
Using 0 dataloader workers
Logging results to runs\detect\package-detection_v1.0.07
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


       6/30         0G      1.565      3.409      1.525          6        320: 100%|██████████| 5/5 [00:01<00:00,  2.85it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]

                   all          4          6    0.00864      0.667     0.0446     0.0231
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 1, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



6 epochs completed in 0.004 hours.
Optimizer stripped from runs\detect\package-detection_v1.0.07\weights\last.pt, 19.1MB
Optimizer stripped from runs\detect\package-detection_v1.0.07\weights\best.pt, 19.1MB

Validating runs\detect\package-detection_v1.0.07\weights\best.pt...
Ultralytics 8.3.173  Python-3.11.13 torch-2.7.1+cpu CPU (12th Gen Intel Core(TM) i7-12700)
YOLO11s summary (fused): 100 layers, 9,413,574 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.22it/s]

                   all          4          6      0.383      0.167     0.0907     0.0863
                person          4          6      0.383      0.167     0.0907     0.0863
Speed: 0.3ms preprocess, 31.2ms inference, 0.0ms loss, 9.4ms postprocess per image


In [12]:
# 학습 결과 및 모델 파일 확인
print("=== 학습 결과 확인 ===")
print(f"학습 이름: {TRAIN_NAME}")

# 학습 결과 객체 확인
if 'results' in locals():
    print(f"학습 완료: {results}")
    print(f"저장 경로: {results.save_dir}")
    
    # results.best 대신 올바른 속성 사용
    try:
        print(f"결과 타입: {type(results)}")
        if hasattr(results, 'results_dict'):
            metrics = results.results_dict
            print(f"성능 지표: {metrics}")
        elif hasattr(results, 'maps'):
            print(f"mAP 점수: {results.maps}")
    except Exception as e:
        print(f"성능 지표 접근 오류: {e}")
    
    # 생성된 모델 파일들 확인
    weights_dir = os.path.join(results.save_dir, "weights")
    if os.path.exists(weights_dir):
        print("\n생성된 모델 파일:")
        for file in os.listdir(weights_dir):
            file_path = os.path.join(weights_dir, file)
            file_size = os.path.getsize(file_path) / 1024 / 1024  # MB
            print(f"  - {file} ({file_size:.2f} MB)")
            
        # best.pt와 last.pt 파일 확인
        best_model = os.path.join(weights_dir, "best.pt")
        last_model = os.path.join(weights_dir, "last.pt")
        
        if os.path.exists(best_model):
            print(f"✅ 최고 성능 모델: {best_model}")
        if os.path.exists(last_model):
            print(f"✅ 마지막 모델: {last_model}")
            
    else:
        print("weights 폴더가 없습니다.")
else:
    print("학습이 아직 완료되지 않았거나 실패했습니다.")

# runs 폴더 전체 확인
print("\n=== runs 폴더 확인 ===")
if os.path.exists("runs"):
    for root, dirs, files in os.walk("runs"):
        level = root.replace("runs", "").count(os.sep)
        indent = "  " * level
        print(f"{indent}{os.path.basename(root)}/")
        for file in files[:5]:  # 파일이 너무 많으면 5개만
            print(f"{indent}  - {file}")
else:
    print("runs 폴더가 존재하지 않습니다.")

=== 학습 결과 확인 ===
학습 이름: package-detection_v1.0.0
학습 완료: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001EC0415BD90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,

In [ ]:
# 학습 결과에서 metrics 추출 및 저장
import json
from datetime import datetime
import time

def create_metrics_json(results, training_config, start_time, end_time):
    """YOLO 학습 결과에서 metrics.json 생성"""
    
    try:
        # 학습 시간 계산
        training_duration = end_time - start_time
        training_time_str = f"{int(training_duration // 3600)}시간 {int((training_duration % 3600) // 60)}분"
        
        # results.csv에서 성능 지표 읽기
        results_csv_path = os.path.join(results.save_dir, 'results.csv')
        
        metrics = {
            "model_info": {
                "model_name": training_config['model_name'],
                "version": training_config['version'],
                "created_date": datetime.now().isoformat()
            },
            "training_config": {
                "epochs": training_config['epochs'],
                "batch_size": training_config['batch'],
                "image_size": training_config['imgsz'],
                "learning_rate": training_config['lr0']
            },
            "performance": {},
            "training_info": {
                "training_time": training_time_str,
                "training_duration_seconds": int(training_duration)
            }
        }
        
        # results.csv 파일이 있다면 성능 지표 추출
        if os.path.exists(results_csv_path):
            import pandas as pd
            
            # CSV 파일 읽기
            df = pd.read_csv(results_csv_path)
            
            # 마지막 에포크(최고 성능) 데이터
            last_row = df.iloc[-1]
            
            # 성능 지표 추가
            metrics["performance"] = {
                "precision": float(last_row.get('metrics/precision(B)', 0)),
                "recall": float(last_row.get('metrics/recall(B)', 0)),
                "mAP50": float(last_row.get('metrics/mAP50(B)', 0)),
                "mAP50_95": float(last_row.get('metrics/mAP50-95(B)', 0)),
                "train_loss": float(last_row.get('train/box_loss', 0)),
                "val_loss": float(last_row.get('val/box_loss', 0))
            }
            
            # 간단한 정확도 계산 (mAP50 기준)
            metrics["accuracy"] = float(last_row.get('metrics/mAP50(B)', 0))
            
        else:
            # CSV 파일이 없다면 기본값
            metrics["performance"] = {
                "precision": 0.0,
                "recall": 0.0, 
                "mAP50": 0.0,
                "mAP50_95": 0.0,
                "train_loss": 0.0,
                "val_loss": 0.0
            }
            metrics["accuracy"] = 0.0
            
        # 데이터셋 정보 (data.yaml에서 추출)
        try:
            with open(data_yaml_path, 'r') as f:
                data_config = yaml.safe_load(f)
            
            # 학습/검증 이미지 수 계산
            train_images = len([f for f in os.listdir(os.path.join(dataset_path, 'train', 'images')) 
                               if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            val_images = len([f for f in os.listdir(os.path.join(dataset_path, 'val', 'images'))
                             if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            
            metrics["dataset_info"] = {
                "num_classes": data_config.get('nc', 0),
                "class_names": data_config.get('names', {}),
                "train_images": train_images,
                "val_images": val_images,
                "total_images": train_images + val_images
            }
            
        except Exception as e:
            print(f"데이터셋 정보 추출 실패: {e}")
            metrics["dataset_info"] = {
                "num_classes": 2,
                "class_names": {0: "person", 1: "package"},
                "train_images": 0,
                "val_images": 0,
                "total_images": 0
            }
        
        # metrics.json 파일 생성
        metrics_path = os.path.join(results.save_dir, 'metrics.json')
        with open(metrics_path, 'w', encoding='utf-8') as f:
            json.dump(metrics, f, indent=2, ensure_ascii=False)
            
        print(f"metrics.json 생성: {metrics_path}")
        return metrics_path, metrics
        
    except Exception as e:
        print(f"metrics.json 생성 실패: {e}")
        return None, None

# 학습 시작 시간 기록
start_time = time.time()

# 모델 학습 (기존 코드)
# ... model.train() ...

# 학습 완료 시간 기록
end_time = time.time()

# metrics.json 생성
if 'results' in locals():
    metrics_path, metrics_data = create_metrics_json(results, training_config, start_time, end_time)
    
    if metrics_path:
        print("\n생성된 성능 지표:")
        print(f"정확도 (mAP50): {metrics_data['accuracy']:.3f}")
        print(f"학습 시간: {metrics_data['training_info']['training_time']}")
        print(f"학습 이미지 수: {metrics_data['dataset_info']['train_images']}")

✅ metrics.json 생성: runs\detect\package-detection_v1.0.07\metrics.json

📊 생성된 성능 지표:
정확도 (mAP50): 0.045
학습 시간: 0시간 0분
학습 이미지 수: 19


In [18]:
# 모델 업로드
from datetime import datetime

def upload_model_to_minio(results, training_config, client):

    # 모델 경로 설정
    model_version = training_config['version']
    model_name = training_config['model_name']
    
    weights_dir = os.path.join(results.save_dir, "weights")
    best_model_path = os.path.join(weights_dir, "best.pt")
    last_model_path = os.path.join(weights_dir, "last.pt")
    
    # MinIO 경로 설정
    minio_model_dir = f"models/training/{model_name}_{model_version}"
    
    print(f"=== MinIO 모델 업로드 시작 ===")
    print(f"모델 버전: {model_name}_{model_version}")
    print(f"로컬 경로: {weights_dir}")
    print(f"MinIO 경로: {minio_model_dir}")
    
    try:
        # 1. best.pt 업로드
        if os.path.exists(best_model_path):
            minio_best_path = f"{minio_model_dir}/best.pt"
            client.fput_object("ai-door-system", minio_best_path, best_model_path)
            
            file_size = os.path.getsize(best_model_path) / 1024 / 1024
            print(f"✅ best.pt 업로드 완료: {file_size:.2f} MB")
        else:
            print("❌ best.pt 파일이 없습니다")
            
        # 2. last.pt 업로드
        if os.path.exists(last_model_path):
            minio_last_path = f"{minio_model_dir}/last.pt"
            client.fput_object("ai-door-system", minio_last_path, last_model_path)
            
            file_size = os.path.getsize(last_model_path) / 1024 / 1024
            print(f"✅ last.pt 업로드 완료: {file_size:.2f} MB")
            
        # 3. 설정 파일 업로드 (config.yaml)
        config_path = "./config.yaml"
        if os.path.exists(config_path):
            minio_config_path = f"{minio_model_dir}/config.yaml"
            client.fput_object("ai-door-system", minio_config_path, config_path)
            print("✅ config.yaml 업로드 완료")
            
        # 4. 메타데이터 생성 및 업로드
        metadata = {
            "model_name": model_name,
            "version": model_version,
            "created_date": datetime.now().isoformat(),
            "training_config": training_config,
            "model_files": {
                "best_model": f"{minio_model_dir}/best.pt",
                "last_model": f"{minio_model_dir}/last.pt",
                "config": f"{minio_model_dir}/config.yaml"
            },
            "local_save_dir": str(results.save_dir)
        }
        
        # 임시 메타데이터 파일 생성
        metadata_path = "/tmp/metadata.json"
        with open(metadata_path, 'w') as f:
            json.dump(metadata, f, indent=2)
            
        # 메타데이터 업로드
        minio_metadata_path = f"{minio_model_dir}/metadata.json"
        client.fput_object("ai-door-system", minio_metadata_path, metadata_path)
        print("✅ metadata.json 업로드 완료")
        
        # 5. 학습 결과 파일들 업로드 (선택적)
        result_files = ['results.png', 'confusion_matrix.png', 'results.csv']
        for file_name in result_files:
            local_file = os.path.join(results.save_dir, file_name)
            if os.path.exists(local_file):
                minio_file_path = f"{minio_model_dir}/{file_name}"
                client.fput_object("ai-door-system", minio_file_path, local_file)
                print(f"✅ {file_name} 업로드 완료")
                
        # 6. metrics.json 업로드 추가
        metrics_file = os.path.join(results.save_dir, 'metrics.json')
        if os.path.exists(metrics_file):
            minio_metrics_path = f"{minio_model_dir}/metrics.json"
            client.fput_object("ai-door-system", minio_metrics_path, metrics_file)
            print("✅ metrics.json 업로드 완료")

        print(f"\n🎉 모델 업로드 완료!")
        print(f"📁 MinIO 경로: ai-door-system/{minio_model_dir}/")
        
        # 업로드된 파일 목록 확인
        print("\n📋 업로드된 파일:")
        objects = client.list_objects("ai-door-system", prefix=f"{minio_model_dir}/", recursive=True)
        for obj in objects:
            print(f"  - {obj.object_name}")
            
        return minio_model_dir
        
    except Exception as e:
        print(f"❌ 업로드 실패: {e}")
        return None

In [19]:
# 학습 완료 후 업로드 실행
if 'results' in locals() and results:
    uploaded_path = upload_model_to_minio(results, training_config, client)
    
    if uploaded_path:
        print(f"\n✅ 모델이 성공적으로 업로드되었습니다!")
        print(f"MinIO 접근 경로: ai-door-system/{uploaded_path}/best.pt")
else:
    print("❌ 학습 결과가 없어 업로드를 건너뜁니다.")

=== MinIO 모델 업로드 시작 ===
모델 버전: package-detection_v1.0.0
로컬 경로: runs\detect\package-detection_v1.0.07\weights
MinIO 경로: models/training/package-detection_v1.0.0
✅ best.pt 업로드 완료: 18.24 MB
✅ last.pt 업로드 완료: 18.24 MB
✅ config.yaml 업로드 완료
✅ metadata.json 업로드 완료
✅ results.csv 업로드 완료
✅ metrics.json 업로드 완료

🎉 모델 업로드 완료!
📁 MinIO 경로: ai-door-system/models/training/package-detection_v1.0.0/

📋 업로드된 파일:
  - models/training/package-detection_v1.0.0/best.pt
  - models/training/package-detection_v1.0.0/config.yaml
  - models/training/package-detection_v1.0.0/last.pt
  - models/training/package-detection_v1.0.0/metadata.json
  - models/training/package-detection_v1.0.0/metrics.json
  - models/training/package-detection_v1.0.0/results.csv

✅ 모델이 성공적으로 업로드되었습니다!
MinIO 접근 경로: ai-door-system/models/training/package-detection_v1.0.0/best.pt
